In [ ]:
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

In [5]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

create dataset, training and testing

In [ ]:
dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train, columns=["text"]))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val, columns=["text"]))
dataset = DatasetDict({"train": dataset_train, "test": dataset_val})

In [ ]:
!huggingface-cli login --token <TODO: CREATE YOUR OWN TOKEN>

In [ ]:
# Reference: https://huggingface.co/blog/4bit-transformers-bitsandbytes
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    quantization_config=bnb_config
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

In [ ]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
# Reference: https://huggingface.co/docs/peft/en/quicktour
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

In [ ]:
# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
    output_dir=,
    num_train_epochs=,
    learning_rate=,
    ...
)

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)
trainer.train()

In [ ]:
# Save the model
trainer.model.save_pretrained("ESE577_chatbot")
model.config.use_cache = True
model.eval()

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INST]@ESE577. {question}. [/INST]"
  return prompt

while True:
  question = input("Enter your ESE577-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()
